In [103]:
import pandas as pd
import numpy as np 
import matplotlib.pyplot as plt
import pytz
import astropy
import sys
import astropy.units as u
from skyfield.api import Topos, load, EarthSatellite
from datetime import datetime, timedelta
from astropy.coordinates import get_body, get_sun, EarthLocation, SkyCoord, AltAz
import lumos.calculator
import lumos.conversions
import json
import os
sys.path.append('/Users/physarah/Development/satellite-optical-brightness/analysis')

import satellite_models.diffuse_sphere as diffuse_sphere
import satellite_models.starlink_v1p5 as starlink_v1p5

def append_record(record, file):
    with open(file, 'a') as f:
        json.dump(record, f)
        f.write(os.linesep)

In [2]:
observer_location = Topos('33.7703 S', '151.1112 E', elevation_m=70)
macquarie_observatory = EarthLocation(lat=-33.7703*u.deg, lon=151.1112*u.deg, height=70*u.m)

sat_cat = "/Users/physarah/Development/satellite-optical-brightness/data/brightness_config_list.csv"
open_list = pd.read_csv(sat_cat)
ts = load.timescale()

In [3]:
cleaned_list = open_list[6:-1]

In [4]:
list_of_sats = cleaned_list['# This file contains a list of NORAD CAT IDs'].to_list()

In [5]:
pos_url = 'https://celestrak.org/NORAD/elements/gp.php?GROUP=starlink&FORMAT=tle'
sats_all = load.tle(pos_url)

In [36]:
def generate_datetime_range(start_time, end_time, interval):
    """
    Generate a list of datetimes between start_time and end_time at a given interval.
    """
    current_time = start_time
    datetime_list = []

    while current_time <= end_time:
        datetime_list.append(current_time)# - timedelta(hours = time_delta))
        current_time += interval

    return datetime_list

start_time = datetime(2024, 1, 9, 0, 0, 1)  
end_time = datetime(2024, 1, 10, 0, 0, 1)   
time_interval = timedelta(minutes=1)          

list_of_times = generate_datetime_range(start_time, end_time, time_interval)
times = []
for timestamp_pd in list_of_times:
    TIME_PD = timestamp_pd.replace(tzinfo=pytz.utc)
    time_sf = ts.utc(TIME_PD)
    times.append(time_sf)

In [70]:
sat_alts_list = []
sat_azs_list = []
sat_names = []
# sun_alitude = []
# sun_azimuth = []
for norad_catid in list_of_sats:
    try:
        sats_catid = sats_all[int(norad_catid)]
        alt_test = []
        az_test = []
        # sun_alt_test = []
        # sun_az_test = []
        for time_now in times:
            topocentric = (sats_catid - observer_location).at(time_now)
            alt, az, _ = topocentric.altaz()  
            alt_test.append(alt.degrees)
            az_test.append(az.degrees)
            # sun = (get_sun(time_now.to_astropy()).icrs).transform_to(AltAz(obstime=time_now.to_astropy(),location=macquarie_observatory)) 
            # sun_alt_test.append(sun.alt.deg)
            # sun_az_test.append(sun.az.deg)
            
        sat_names.append(norad_catid)
        # sun_azimuth.append(sun_az_test)
        # sun_alitude.append(sun_alt_test)
        sat_alts_list.append(alt_test)
        sat_azs_list.append(az_test)
    except:
        pass    

In [38]:
def count_negative_to_positive_transitions(alts):
    transition_count = 0

    for i in range(1, len(alts)):
        if alts[i-1] < 0 and alts[i] > 0:
            transition_count += 1

    return transition_count

result = count_negative_to_positive_transitions(sat_alts_list[0])
print(f"Negative to positive transitions: {result}")

Negative to positive transitions: 4


In [105]:
for j in np.arange(0,len(sat_names[0])):
    
    cat_sat_name = sat_names[j]
    sat_height = np.array([(570 + 540)/2]*len(times) * 1000)
    satellite_heights = sat_height
    observation_times = [dt.utc_iso().split('.') for dt in times]
    satellite_altitudes = sat_alts_list[j]
    satellite_azimuths = sat_azs_list[j]
    measured_magnitudes = np.random.rand(len(satellite_azimuths))
    
    intensities_diffuse_sphere = np.zeros_like(measured_magnitudes)
    intensities_lab_brdfs = np.zeros_like(measured_magnitudes)
    intensities_inferred_brdfs = np.zeros_like(measured_magnitudes)
    
    sun_alt_list, sun_az_list = ([],[])
    for i, (sat_alt, sat_az, sat_h, time) in \
        enumerate(zip(satellite_altitudes, satellite_azimuths, satellite_heights, observation_times)):
        
        observation_time = astropy.time.Time(time, format = 'isot')
        sun_alt, sun_az = lumos.calculator.get_sun_alt_az(observation_time, macquarie_observatory)
        sun_alt_list.append(sun_alt)
        sun_az_list.append(sun_az)

In [107]:
for j in np.arange(0,len(sat_names)):
    
    cat_sat_name = sat_names[j]
    sat_height = np.array([(570 + 540)/2]*len(times) * 1000)
    satellite_heights = sat_height
    observation_times = [dt.utc_iso().split('.') for dt in times]
    satellite_altitudes = sat_alts_list[j]
    satellite_azimuths = sat_azs_list[j]
    measured_magnitudes = np.random.rand(len(satellite_azimuths))
    
    intensities_lab_brdfs = np.zeros_like(measured_magnitudes)
    
    for i, (sat_alt, sat_az, sat_h, time, sun_alt, sun_az) in \
        enumerate(zip(satellite_altitudes, satellite_azimuths, 
                      satellite_heights, observation_times,
                     [item[0] for item in sun_alt_list],
                     [item[0] for item in sun_az_list])):
        
        observation_time = astropy.time.Time(time, format = 'isot')
        
        intensities_lab_brdfs[i] = \
            lumos.calculator.get_intensity_observer_frame(
                starlink_v1p5.SURFACES_LAB_BRDFS,
                sat_h, sat_alt, sat_az, sun_alt, sun_az,
                include_earthshine = True,
                earth_brdf=lumos.brdf.library.PHONG(0.53, 0.28, 7.31)) # use the vegitation example that Forrest uses 
    
    magnitudes_lab_brdfs = lumos.conversions.intensity_to_ab_mag(intensities_lab_brdfs)

    big_dict = {'sun_altitude':[item[0] for item in sun_alt_list],
                'sun_azimuth':[item[0] for item in sun_az_list],
                'magnitude':list(magnitudes_lab_brdfs),
                'sat_altitude':satellite_altitudes,
                'sat_azimuth':satellite_azimuths,
                'times':[item[0] for item in observation_times],
                'cat_name':[cat_sat_name]*len(observation_times)}
    

    append_record(big_dict, '/Users/physarah/Desktop/sat_paths.json')

KeyboardInterrupt: 